# Features extraction - radiomics

#### Note: run inside an environment with the most recent version of numpy and pyradiomics

In [1]:
import os
import six
import pickle
import pandas as pd
import radiomics
import SimpleITK as sitk
from radiomics import featureextractor
from functools import reduce
import numpy as np
from skimage.measure import find_contours
import pylidc as pl
from pylidc.utils import consensus
import matplotlib.pyplot as plt

### 2D features extration

#### Configuração do ficheiro Params.yaml

In [2]:
#Settings to use, possible settings are listed in the documentation (section "Customizing the extraction").
setting:
  binWidth: 25
  label: 1
  interpolator: 'sitkBSpline' # This is an enumerated value, here None is not allowed
  resampledPixelSpacing: # This disables resampling, as it is interpreted as None, to enable it, specify spacing in x, y, z as [x, y , z]
  #verbose: True
  weightingNorm: # If no value is specified, it is interpreted as None
  force2D: True  # Esta configuração ativa a extração de características 2D

#Image types to use: "Original" for unfiltered image, for possible filters, see documentation.
imageType:
  Original: {} # for dictionaries / mappings, None values are not allowed, '{}' is interpreted as an empty dictionary

#Featureclasses, from which features must be calculated. If a featureclass is not mentioned, no features are calculated
#for that class. Otherwise, the specified features are calculated, or, if none are specified, all are calculated (excluding redundant/deprecated features).
featureClass:
  shape:
  firstorder: [] # specifying an empty list has the same effect as specifying nothing.
  glcm:  # Disable SumAverage by specifying all other GLCM features available
    - 'Autocorrelation'
    - 'JointAverage'
    - 'ClusterProminence'
    - 'ClusterShade'
    - 'ClusterTendency'
    - 'Contrast'
    - 'Correlation'
    - 'DifferenceAverage'
    - 'DifferenceEntropy'
    - 'DifferenceVariance'
    - 'JointEnergy'
    - 'JointEntropy'
    - 'Imc1'
    - 'Imc2'
    - 'Idm'
    - 'Idmn'
    - 'Id'
    - 'Idn'
    - 'InverseVariance'
    - 'MaximumProbability'
    - 'SumEntropy'
    - 'SumSquares'
  glrlm: # for lists none values are allowed, in this case, all features are enabled
  glszm:
  gldm:  # contains deprecated features, but as no individual features are specified, the deprecated features are not enabled


SyntaxError: invalid syntax (<ipython-input-2-c54755b6696e>, line 2)

#### std_limit = 0.5

In [3]:
# Path were the images are stored
input_directory = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/LIDC-IDRI"

# Path to the setup file from radiomics
params_file = "/home/cmonteiro/pyradiomics-master/pyradiomics-master/examples/exampleSettings/Params.yaml"

path = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/std_limit_0.5"

extractor = featureextractor.RadiomicsFeatureExtractor(params_file)

dic_list = []

# List of all the archives on the external drive
archives_npy = [archive for archive in os.listdir(path) if archive.endswith('cmask.npy')]

# Order the archives
ordered_archives = sorted(archives_npy, key=lambda x: int(x.split('_')[0]))

dic_list = []

for archive in ordered_archives:
    cmask = np.load(os.path.join(path, archive))
    image = sitk.GetImageFromArray(cmask.astype(float))

    features = extractor.execute(image, image, label=1)
    dic_list.append(features)

    nodule_id = int(archive.split('_')[0])
    features['Nodule_id'] = nodule_id

features = pd.DataFrame(dic_list)
csv_filename = os.path.join(path, 'features_radiomics_2d_0.5.csv')
features.to_csv(csv_filename, index=False)
features

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,Nodule_id
0,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},6c9a580a252d49720868291e8202f6280702e325,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,602.3370754981757,602.3370754981757,602.3370754981757,1.0,0.002383863821909873,0.002383863821909873,0.002383863821909873,2
1,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ea87a2a43bba24b4642a0005ebf9ca11496f042c,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,464.24547600314713,464.24547600314713,464.24547600314713,1.0,0.003581554615362348,0.003581554615362348,0.003581554615362348,3
2,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},87deef3afa40c86ea2a0f2f9220b32301bbddfd3,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,268.38129496402877,268.38129496402877,268.38129496402877,1.0,0.006325852130783044,0.006325852130783044,0.006325852130783044,9
3,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5545209080d87a6226708c3ff5713bfd19e32633,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,139.6984126984127,139.6984126984127,139.6984126984127,1.0,0.013553511705536288,0.013553511705536288,0.013553511705536288,10
4,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d15fc136149bf4d2c17594e6a293dc62a78372dc,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,191.10752688172042,191.10752688172042,191.10752688172042,1.0,0.009444693794781118,0.009444693794781118,0.009444693794781118,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},f6c599a2010a58190c6c21a48e99febc140f5fc0,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,192.85714285714286,192.85714285714286,192.85714285714286,1.0,0.007425139351083223,0.007425139351083223,0.007425139351083223,2642
973,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},9fdddde76941458c76678a677dc6cefd38f62ea0,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,162.43589743589743,162.43589743589743,162.43589743589743,1.0,0.008700098304641734,0.008700098304641734,0.008700098304641734,2644
974,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},c13756009931334f56e84d94771265e217f289d5,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,558.2215096201282,558.2215096201282,558.2215096201282,1.0,0.0028782594931771465,0.0028782594931771465,0.0028782594931771465,2645
975,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ca9d8b3a24039e84ad9ec11b01ffca322707e5be,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,204.8,204.8,204.8,1.0,0.006772373496114599,0.006772373496114599,0.006772373496114599,2646


#### std_limit = 1.0

In [4]:
# Path were the images are stored
input_directory = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/LIDC-IDRI"

# Path to the setup file from radiomics
params_file = "/home/cmonteiro/pyradiomics-master/pyradiomics-master/examples/exampleSettings/Params.yaml"

path = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/std_limit_1.0"

extractor = featureextractor.RadiomicsFeatureExtractor(params_file)

dic_list = []

# List of all the archives on the external drive
archives_npy = [archive for archive in os.listdir(path) if archive.endswith('cmask.npy')]

# Order the archives
ordered_archives = sorted(archives_npy, key=lambda x: int(x.split('_')[0]))

dic_list = []

for archive in ordered_archives:
    cmask = np.load(os.path.join(path, archive))
    image = sitk.GetImageFromArray(cmask.astype(float))

    features = extractor.execute(image, image, label=1)
    dic_list.append(features)

    nodule_id = int(archive.split('_')[0])
    features['Nodule_id'] = nodule_id

features = pd.DataFrame(dic_list)
csv_filename = os.path.join(path, 'features_radiomics_2d_1.0.csv')
features.to_csv(csv_filename, index=False)
features


,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,Nodule_id
0,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},6c9a580a252d49720868291e8202f6280702e325,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,602.3370754981757,602.3370754981757,602.3370754981757,1.0,0.002383863821909873,0.002383863821909873,0.002383863821909873,2
1,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ea87a2a43bba24b4642a0005ebf9ca11496f042c,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,464.24547600314713,464.24547600314713,464.24547600314713,1.0,0.003581554615362348,0.003581554615362348,0.003581554615362348,3
2,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},87deef3afa40c86ea2a0f2f9220b32301bbddfd3,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,268.38129496402877,268.38129496402877,268.38129496402877,1.0,0.006325852130783044,0.006325852130783044,0.006325852130783044,9
3,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5545209080d87a6226708c3ff5713bfd19e32633,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,139.6984126984127,139.6984126984127,139.6984126984127,1.0,0.013553511705536288,0.013553511705536288,0.013553511705536288,10
4,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d15fc136149bf4d2c17594e6a293dc62a78372dc,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,191.10752688172042,191.10752688172042,191.10752688172042,1.0,0.009444693794781118,0.009444693794781118,0.009444693794781118,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},f6c599a2010a58190c6c21a48e99febc140f5fc0,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,192.85714285714286,192.85714285714286,192.85714285714286,1.0,0.007425139351083223,0.007425139351083223,0.007425139351083223,2642
1652,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},9fdddde76941458c76678a677dc6cefd38f62ea0,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,162.43589743589743,162.43589743589743,162.43589743589743,1.0,0.008700098304641734,0.008700098304641734,0.008700098304641734,2644
1653,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},c13756009931334f56e84d94771265e217f289d5,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,558.2215096201282,558.2215096201282,558.2215096201282,1.0,0.0028782594931771465,0.0028782594931771465,0.0028782594931771465,2645
1654,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ca9d8b3a24039e84ad9ec11b01ffca322707e5be,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,204.8,204.8,204.8,1.0,0.006772373496114599,0.006772373496114599,0.006772373496114599,2646


#### std_limit = 1.5

In [5]:
# Path were the images are stored
input_directory = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/LIDC-IDRI"

# Path to the setup file from radiomics
params_file = "/home/cmonteiro/pyradiomics-master/pyradiomics-master/examples/exampleSettings/Params.yaml"

path = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/std_limit_1.5"

extractor = featureextractor.RadiomicsFeatureExtractor(params_file)

dic_list = []

# List of all the archives on the external drive
archives_npy = [archive for archive in os.listdir(path) if archive.endswith('cmask.npy')]

# Order the archives
ordered_archives = sorted(archives_npy, key=lambda x: int(x.split('_')[0]))

dic_list = []

for archive in ordered_archives:
    cmask = np.load(os.path.join(path, archive))
    image = sitk.GetImageFromArray(cmask.astype(float))

    features = extractor.execute(image, image, label=1)
    dic_list.append(features)

    nodule_id = int(archive.split('_')[0])
    features['Nodule_id'] = nodule_id

features = pd.DataFrame(dic_list)
csv_filename = os.path.join(path, 'features_radiomics_2d_1.5.csv')
features.to_csv(csv_filename, index=False)
features


,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,Nodule_id
0,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},3dbcca2f07a28dd238aeea62535e787abbfeda44,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,541.6481208548269,541.6481208548269,541.6481208548269,1.0,0.002895684395025496,0.002895684395025496,0.002895684395025496,1
1,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},6c9a580a252d49720868291e8202f6280702e325,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,602.3370754981757,602.3370754981757,602.3370754981757,1.0,0.002383863821909873,0.002383863821909873,0.002383863821909873,2
2,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ea87a2a43bba24b4642a0005ebf9ca11496f042c,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,464.24547600314713,464.24547600314713,464.24547600314713,1.0,0.003581554615362348,0.003581554615362348,0.003581554615362348,3
3,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},8a8f88ba678db54be433931d9507c7b26f4e4f17,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,529.2375809935205,529.2375809935205,529.2375809935205,1.0,0.0026740864264331197,0.0026740864264331197,0.0026740864264331197,4
4,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ccaf09e8511dca39e32d79a8e9e9dd856472b0ff,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,335.911877394636,335.911877394636,335.911877394636,1.0,0.004936442450904336,0.004936442450904336,0.004936442450904336,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},c13756009931334f56e84d94771265e217f289d5,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,558.2215096201282,558.2215096201282,558.2215096201282,1.0,0.0028782594931771465,0.0028782594931771465,0.0028782594931771465,2645
2436,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ca9d8b3a24039e84ad9ec11b01ffca322707e5be,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,204.8,204.8,204.8,1.0,0.006772373496114599,0.006772373496114599,0.006772373496114599,2646
2437,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},aa28668ac61b24d0fb8627df9fab91ddcf0843f0,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,506.09939337377506,506.09939337377506,506.09939337377506,1.0,0.0030115297485682215,0.0030115297485682215,0.0030115297485682215,2648
2438,v3.1.0,1.23.5,2.3.0,1.1.1,3.9.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},347615a17f395f599e2ed65b6c358a55f4dc744e,3D,"(1.0, 1.0, 1.0)",...,0.0,1.0,438.26909722222223,438.26909722222223,438.26909722222223,1.0,0.003724504579796317,0.003724504579796317,0.003724504579796317,2649


### 3D features extration

#### Due to the lack of storage this part of the code can not be executed 

#### Configuração do ficheiro Params.yaml

In [ ]:
#Settings to use, possible settings are listed in the documentation (section "Customizing the extraction").
setting:
  binWidth: 25
  label: 1
  interpolator: 'sitkBSpline' # This is an enumerated value, here None is not allowed
  resampledPixelSpacing: # This disables resampling, as it is interpreted as None, to enable it, specify spacing in x, y, z as [x, y , z]
  #verbose: True
  weightingNorm: # If no value is specified, it is interpreted as None
  force2D: False  # Esta configuração ativa a extração de características 2D

#Image types to use: "Original" for unfiltered image, for possible filters, see documentation.
imageType:
  Original: {} # for dictionaries / mappings, None values are not allowed, '{}' is interpreted as an empty dictionary

#Featureclasses, from which features must be calculated. If a featureclass is not mentioned, no features are calculated
#for that class. Otherwise, the specified features are calculated, or, if none are specified, all are calculated (excluding redundant/deprecated features).
featureClass:
  shape:
  firstorder: [] # specifying an empty list has the same effect as specifying nothing.
  glcm:  # Disable SumAverage by specifying all other GLCM features available
    - 'Autocorrelation'
    - 'JointAverage'
    - 'ClusterProminence'
    - 'ClusterShade'
    - 'ClusterTendency'
    - 'Contrast'
    - 'Correlation'
    - 'DifferenceAverage'
    - 'DifferenceEntropy'
    - 'DifferenceVariance'
    - 'JointEnergy'
    - 'JointEntropy'
    - 'Imc1'
    - 'Imc2'
    - 'Idm'
    - 'Idmn'
    - 'Id'
    - 'Idn'
    - 'InverseVariance'
    - 'MaximumProbability'
    - 'SumEntropy'
    - 'SumSquares'
  glrlm: # for lists none values are allowed, in this case, all features are enabled
  glszm:
  gldm:  # contains deprecated features, but as no individual features are specified, the deprecated features are not enabled


#### std_limit = 0.5

In [ ]:
# Path were the images are stored
input_directory = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/LIDC-IDRI"

# Path to the setup file from radiomics
params_file = "/home/cmonteiro/pyradiomics-master/pyradiomics-master/examples/exampleSettings/Params.yaml"

path = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/std_limit_0.5"

extractor = featureextractor.RadiomicsFeatureExtractor(params_file)

dic_list = []

# List of all the archives on the external drive
archives_npy = [archive for archive in os.listdir(path) if archive.endswith('volume.npy')]

# Order the archives
ordered_archives = sorted(archives_npy, key=lambda x: int(x.split('_')[0]))

dic_list = []

for archive in ordered_archives:
    volume = np.load(os.path.join(path, archive))
    image = sitk.GetImageFromArray(volume.astype(float))

    features = extractor.execute(image, image, label=1)
    dic_list.append(features)

    nodule_id = int(archive.split('_')[0])
    features['Nodule_id'] = nodule_id

features = pd.DataFrame(dic_list)
csv_filename = os.path.join(path, 'features_radiomics_3d_0.5.csv')
features.to_csv(csv_filename, index=False)
features


#### std_limit = 1.0

In [ ]:
# Path were the images are stored
input_directory = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/LIDC-IDRI"

# Path to the setup file from radiomics
params_file = "/home/cmonteiro/pyradiomics-master/pyradiomics-master/examples/exampleSettings/Params.yaml"

path = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/std_limit_1.0"

extractor = featureextractor.RadiomicsFeatureExtractor(params_file)

dic_list = []

# List of all the archives on the external drive
archives_npy = [archive for archive in os.listdir(path) if archive.endswith('volume.npy')]

# Order the archives
ordered_archives = sorted(archives_npy, key=lambda x: int(x.split('_')[0]))

dic_list = []

for archive in ordered_archives:
    volume = np.load(os.path.join(path, archive))
    image = sitk.GetImageFromArray(volume.astype(float))

    features = extractor.execute(image, image, label=1)
    dic_list.append(features)

    nodule_id = int(archive.split('_')[0])
    features['Nodule_id'] = nodule_id

features = pd.DataFrame(dic_list)
csv_filename = os.path.join(path, 'features_radiomics_3d_1.0.csv')
features.to_csv(csv_filename, index=False)
features


#### std_limit = 1.5

In [ ]:
# Path were the images are stored
input_directory = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/LIDC-IDRI"

# Path to the setup file from radiomics
params_file = "/home/cmonteiro/pyradiomics-master/pyradiomics-master/examples/exampleSettings/Params.yaml"

path = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/std_limit_1.5"

extractor = featureextractor.RadiomicsFeatureExtractor(params_file)

dic_list = []

# List of all the archives on the external drive
archives_npy = [archive for archive in os.listdir(path) if archive.endswith('volume.npy')]

# Order the archives
ordered_archives = sorted(archives_npy, key=lambda x: int(x.split('_')[0]))

dic_list = []

for archive in ordered_archives:
    volume = np.load(os.path.join(path, archive))
    image = sitk.GetImageFromArray(volume.astype(float))

    features = extractor.execute(image, image, label=1)
    dic_list.append(features)

    nodule_id = int(archive.split('_')[0])
    features['Nodule_id'] = nodule_id

features = pd.DataFrame(dic_list)
csv_filename = os.path.join(path, 'features_radiomics_3d_1.5.csv')
features.to_csv(csv_filename, index=False)
features
